## CAI - Rebalancing data for October 2023 

## Configuring packages

In [1]:
import pandas as pd
import numpy as np
import requests
import decouple
import sys 
import datetime
import time
sys.path.append('../')

from web3 import Web3
from abis import index_anatomy


infura_url = decouple.config("AVALANCHE_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

import db_funcs

In [2]:
from pycoingecko import CoinGeckoAPI
key = decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key= key)

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables and settings

In [5]:
db_key_date = '2023-10-01'
db_liquidity_table = 'cai_liquidities'
db_benchmark_table = 'cai_benchmark_data'
min_weight = 0.005
max_weight = 0.5
supply_threshold = 0.3
liquidity_consistency = 90 # In days
avax_to_savax_split = 0.6
liveness_threshold = 30*6 # In days

In [6]:
liq = 2*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.09

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data at least 6 months prior to the date of inclusion in the index.
    1. The project should have a token that is native to Avalanche. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project.
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. Projects that have ponzi characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible.
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
    <br>
1. **Pricing requirements**
    1. Token should have a Chainlink price feed to allow for manipulation resistant pricing.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange. 
    1. The token should have in aggregate at least $2mm of onchain liquidity across Trader Joe, Platypus, Pangolin.
    1. The token must have shown consistent DeFi liquidity on Avalanche.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [7]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### CoinGecko

##### Tokens addresses

In [8]:
coins_list = cg.get_coins_list(include_platform=True)
coins = {}
for coin in coins_list:
    if 'avalanche' in coin['platforms']:
        coin['addr'] = coin['platforms']['avalanche']
        coin['symbol'] = normalize_symb(coin['symbol'])
        coins[coin['id']] = coin

# Adding AVAX
coins['avalanche-2'] = {'id': 'avalanche-2','symbol':normalize_symb('avax'), 'name':'Avalanche','addr':'0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7',}


##### Market data

In [9]:
cat_id = "avalanche-ecosystem"
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id)
for i in range(2, 30):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', category=cat_id, page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)


#print(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'))

cg_tokens_data = pd.concat([cg_tokens_data,pd.DataFrame(cg.get_coins_markets(ids='avalanche-2',vs_currency='USD'))],ignore_index=True)
cg_tokens_data

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.000000,8.328307e+10,3.0,8.328307e+10,2.365716e+10,1.005000,...,8.325705e+10,NaN,1.320000,-24.40131,2018-07-24T00:00:00.000Z,0.572521,74.70847,2015-03-02T00:00:00.000Z,None,2023-10-02T11:35:00.447Z
1,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,219.350000,3.377782e+10,4.0,4.390832e+10,5.449671e+08,221.090000,...,1.538562e+08,2.000000e+08,686.310000,-68.04030,2021-05-10T07:24:17.097Z,0.039818,550764.41614,2017-10-19T00:00:00.000Z,None,2023-10-02T11:35:19.100Z
2,usd-coin,usdc,USDC,https://assets.coingecko.com/coins/images/6319...,0.999924,2.527049e+10,6.0,2.527034e+10,6.311906e+09,1.004000,...,2.526733e+10,NaN,1.170000,-14.62241,2019-05-08T00:40:28.300Z,0.877647,14.08102,2023-03-11T08:02:13.981Z,None,2023-10-02T11:35:23.901Z
3,wrapped-bitcoin,wbtc,Wrapped Bitcoin,https://assets.coingecko.com/coins/images/7598...,28337.000000,4.617194e+09,17.0,4.617194e+09,1.572964e+08,28386.000000,...,1.627949e+05,1.627949e+05,70643.000000,-59.90771,2021-11-10T14:40:19.650Z,3139.170000,802.22985,2019-04-02T00:00:00.000Z,None,2023-10-02T11:35:16.325Z
4,chainlink,link,Chainlink,https://assets.coingecko.com/coins/images/877/...,7.930000,4.409911e+09,19.0,7.919388e+09,3.818976e+08,8.160000,...,1.000000e+09,1.000000e+09,52.700000,-84.95357,2021-05-10T00:13:57.214Z,0.148183,5250.77861,2017-11-29T00:00:00.000Z,None,2023-10-02T11:35:22.800Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,ethereum-wormhole,eth,Ethereum (Wormhole),https://assets.coingecko.com/coins/images/2299...,1728.520000,0.000000e+00,NaN,1.408378e+08,1.197975e+06,1754.600000,...,8.144752e+04,NaN,3892.040000,-55.50294,2022-03-30T16:00:34.289Z,146.450000,1082.53010,2022-10-10T21:11:53.498Z,None,2023-10-02T11:35:00.256Z
327,evoverses,evo,EvoVerses,https://assets.coingecko.com/coins/images/2508...,0.000392,0.000000e+00,NaN,1.079570e+05,1.165100e+02,0.000394,...,2.754408e+08,NaN,0.034352,-98.85903,2022-07-20T07:01:41.657Z,0.000357,9.75771,2023-08-11T16:46:09.736Z,None,2023-10-01T14:09:16.245Z
328,zjoe,zjoe,zJOE,https://assets.coingecko.com/coins/images/2553...,0.163060,0.000000e+00,NaN,NaN,2.684000e+01,0.169809,...,NaN,NaN,0.467473,-65.06901,2022-05-18T01:29:44.179Z,0.087044,87.59913,2023-03-11T15:21:46.299Z,None,2023-10-02T11:35:19.092Z
329,backed-niu-technologies,bniu,Backed NIU Technologies,https://assets.coingecko.com/coins/images/3186...,NaN,NaN,NaN,NaN,NaN,NaN,...,1.600960e+05,NaN,NaN,0.00000,None,NaN,0.00000,None,None,None


In [10]:
tokens_market_data = cg_tokens_data.set_index('id')[
    ['name', 'symbol', 'market_cap', 'market_cap_rank', 'current_price', 'total_supply', 'circulating_supply']
]
tokens_market_data['symbol'] = tokens_market_data['symbol'].apply(normalize_symb)

In [11]:
tokens_market_data = tokens_market_data[tokens_market_data['market_cap'] > 0]
tokens_market_data = tokens_market_data[~tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.join(pd.DataFrame.from_dict(coins, orient='index')['addr'])
tokens_market_data = tokens_market_data[~tokens_market_data['addr'].isna()]


In [12]:
# Stable coins to remove from tokens list
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')

#Adjusting Pangolin's max supply to reflect tokenomic changes here: https://app.pangolin.exchange/#/vote/10
tokens_market_data.loc[['pangolin'],["total_supply"]] = 230000000.0

tokens_market_data =  tokens_market_data[~tokens_market_data.index.isin(stablecoins.index)]
tokens_market_data = tokens_market_data.sort_values('market_cap', ascending=False)

tokens_market_data

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
wrapped-bitcoin,Wrapped Bitcoin,WBTC,4.617194e+09,17.0,28337.000000,1.627949e+05,1.627949e+05,0x50b7545627a5162f82a992c33b87adc75187b218
chainlink,Chainlink,LINK,4.409911e+09,19.0,7.930000,1.000000e+09,5.568500e+08,0x5947bb275c521040051d82396192181b413227a3
uniswap,Uniswap,UNI,3.514800e+09,21.0,4.660000,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
avalanche-2,Avalanche,AVAX,3.450144e+09,23.0,9.730000,4.315704e+08,3.541961e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
maker,Maker,MKR,1.344232e+09,38.0,1490.550000,9.776310e+05,9.013109e+05,0x88128fd4b259552a9a1d457f435a6527aab72d42
...,...,...,...,...,...,...,...,...
ydragon,YDragon,YDR,7.752890e+03,3509.0,0.000062,5.000000e+08,1.257806e+08,0xf03dccaec9a28200a6708c686cf0b8bf26ddc356
hakuswap,HakuSwap,HAKU,7.259450e+03,3514.0,0.000167,8.452850e+07,4.343500e+07,0x695fa794d59106cebd40ab5f5ca19f458c723829
echidna,Echidna,ECD,3.263040e+03,3556.0,0.000284,3.383739e+08,1.149388e+07,0xeb8343d5284caec921f035207ca94db6baaacbcd


### Token's supply check 

In [13]:
sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > supply_threshold
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & token age check

In [14]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id_, row in sp_passed.iterrows():
    symbol = row['symbol']
    data = cg.get_coin_market_chart_by_id(id_, vs_currency='USD', days='max')
  
    df_prices = pd.DataFrame(data['prices'], columns=['date', id_])
    df_prices = df_prices[df_prices[id_] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < liveness_threshold:
        print(f'Excluding {symbol}, prices data available only for {len(df_prices)} < {liveness_threshold} days')
        exclude_list.append(id_)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id_])
    df_mcaps = df_mcaps[df_mcaps[id_] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < liveness_threshold:
        print(f'Note: {symbol}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    
    

Excluding MELD, prices data available only for 148 < 180 days
Excluding USP, prices data available only for 13 < 180 days
Excluding ERN, prices data available only for 176 < 180 days
Excluding 3ULL, prices data available only for 116 < 180 days
Note: EIV, marketcap data available only for 47 < 180 days
Note: EUROE, marketcap data available only for 103 < 180 days
Note: GMD, marketcap data available only for 125 < 180 days
Note: CINU, marketcap data available only for 165 < 180 days
Excluding GRAIN, prices data available only for 172 < 180 days
Note: PLN, marketcap data available only for 125 < 180 days
Note: VCHF, marketcap data available only for 3 < 180 days
Note: NFTE, marketcap data available only for 12 < 180 days


### Non quantative checks

In [15]:
manual_exclusions = [
    {
        'gecko_id': 'spell-token',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'stacktical',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'zookeeper',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'pendle',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'everrise',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason' : 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'bitcoin-avalanche-bridged-btc-b',
        'reason': 'Not a native Avalanche asset'
    },
    {
        'gecko_id': 'gmx',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Bridged asset'
    },
    {
        'gecko_id': 'euro-coin',
        'reason': 'Stable coin'
    }
]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])

In [16]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list')
passed_q_and_non_q_checks.drop(passed_q_and_non_q_checks[passed_q_and_non_q_checks['market_cap'] < 1e6].index)

,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
chainlink,Chainlink,LINK,4.409911e+09,19.0,7.930000e+00,1.000000e+09,5.568500e+08,0x5947bb275c521040051d82396192181b413227a3
uniswap,Uniswap,UNI,3.514800e+09,21.0,4.660000e+00,1.000000e+09,7.537667e+08,0x8ebaf22b6f053dffeaf46f4dd9efa95d89ba8580
avalanche-2,Avalanche,AVAX,3.450144e+09,23.0,9.730000e+00,4.315704e+08,3.541961e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
maker,Maker,MKR,1.344232e+09,38.0,1.490550e+03,9.776310e+05,9.013109e+05,0x88128fd4b259552a9a1d457f435a6527aab72d42
aave,Aave,AAVE,1.024344e+09,45.0,7.014000e+01,1.600000e+07,1.457929e+07,0x63a72806098bd3d9520cc43356dd78afe5d386d9
the-graph,The Graph,GRT,8.525801e+08,47.0,9.232700e-02,1.000000e+10,9.235807e+09,0x8a0cac13c7da965a312f08ea4229c37869e85cb9
havven,Synthetix Network,SNX,6.984660e+08,55.0,2.150000e+00,3.248085e+08,3.240846e+08,0xbec243c995409e6520d7c41e404da5deba4b209b
frax-share,Frax Share,FXS,4.208120e+08,88.0,5.660000e+00,9.968150e+07,7.424780e+07,0x214db107654ff987ad859f34125307783fc8e387
compound-governance-token,Compound,COMP,3.297931e+08,107.0,4.806000e+01,1.000000e+07,6.856086e+06,0xc3048e19e76cb9a3aa9d77d8c03c29fc906e2437


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [17]:
url_0x = "https://avalanche.api.0x.org/swap/v1/quote"
header =  {'0x-api-key': decouple.config("ZEROEX_KEY")}

In [18]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    try:
        query = {
            'buyToken': data['addr'],
            'sellToken': 'AVAX',
            'sellAmount': int(trade_value_tiny / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / tokens_market_data.loc['avalanche-2']['current_price'] * 10 ** decimals)
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        print(id)
        continue
    

havven
balancer
axelar
cartesi
ankreth
allianceblock-nexera
idia
realfevr
oath
euroe-stablecoin
build
spherium
vnx-swiss-franc
soul-swap
shack
bios
nftearth


In [19]:
slippages_df = pd.DataFrame(slippages).sort_values('slippage',ascending=False).reset_index(drop=True)
slippages_df.head(20)

,id,spot_price,delivery_price,slippage
0,avalanche-2,1.000000,1.000000,0.000000
1,benqi-liquid-staked-avax,0.903940,0.903792,-0.000164
2,joe,37.065786,36.306841,-0.020476
3,colony-avalanche-index,0.128656,0.118403,-0.079693
4,benqi,1800.155099,1611.545564,-0.104774
5,betswirl,18093.218639,16165.971705,-0.106518
6,pangolin,498.266149,404.900102,-0.187382
7,avalaunch,57.270288,46.493730,-0.188170
8,chainlink,1.224377,0.959853,-0.216048
9,yield-yak,0.035959,0.020152,-0.439584


In [20]:
# Writing to database
slippages_to_save = slippages_df[['id','slippage']].head(20)
asset_columns = db_funcs.convert_to_sql_strings(list(slippages_to_save['id']))
slippage_values = list(slippages_to_save['slippage'])
db_funcs.insert_values(db_key_date,asset_columns,slippage_values,db_liquidity_table)


In [21]:
def check_avg_liquidity(date_delta):
    start_date = str(datetime.date.fromisoformat(db_key_date) - datetime.timedelta(days=date_delta))
    dataframe = pd.read_sql(f'Select * from {db_liquidity_table} where date >= ? ',db_funcs.create_connection(db_funcs.db),index_col='date',parse_dates=['date'],params=[start_date])
    dataframe = dataframe.ewm(span=3).mean().iloc[-1]
    dataframe = dataframe[dataframe > max_slippage]
    dataframe.index = db_funcs.convert_from_sql_strings(list(dataframe.index))
    return dataframe, list(dataframe.index)

avg_liq_dataframe ,include_list = check_avg_liquidity(liquidity_consistency)
avg_liq_dataframe


benqi-liquid-staked-avax   -0.000155
avalanche-2                 0.000000
joe                        -0.020915
Name: 2023-10-01 00:00:00, dtype: float64

In [22]:
qualified_assets = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets


,name,symbol,market_cap,market_cap_rank,current_price,total_supply,circulating_supply,addr
id,,,,,,,,
avalanche-2,Avalanche,AVAX,3.450144e+09,23.0,9.730000,4.315704e+08,3.541961e+08,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,8.938757e+07,255.0,0.261539,4.764376e+08,3.414594e+08,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,7.074822e+07,319.0,10.770000,6.561828e+06,6.561828e+06,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


### Marketcap ranking & filtering

In [23]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 50%.
* All excess weight is proportionally redistributed to all uncapped tokens. 
* Any asset with a weight below 0.5% will be removed from the index.


### Weight mcaps

In [24]:
# exclude savax from weighting since it should be a part of avax weight
if 'benqi-liquid-staked-avax' in chosen_tokens.index:
    chosen_tokens_ex_savax = chosen_tokens.drop('benqi-liquid-staked-avax')
    weights = chosen_tokens_ex_savax['market_cap'].div(chosen_tokens_ex_savax['market_cap'].sum())
else:
    weights = chosen_tokens['market_cap'].div(chosen_tokens['market_cap'].sum())



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [25]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum()
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum()).mul(c)
    
    return w

In [26]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum())
    
    return w

In [27]:
adjusted_weights = adjust_weights(weights, max_weight=max_weight) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=min_weight)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=max_weight)

### Split AVAX 

In [28]:
if 'avalanche-2' and 'benqi-liquid-staked-avax' in chosen_tokens.index:
    
    adjusted_weights['benqi-liquid-staked-avax'] = adjusted_weights['avalanche-2'] * avax_to_savax_split
    adjusted_weights['avalanche-2'] = adjusted_weights['avalanche-2'] * (1 - avax_to_savax_split)

adjusted_weights =  adjusted_weights.sort_values(ascending = False)


### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [29]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<128]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [30]:
converted_weights = convert_weights(adjusted_weights)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_6932/3538882253.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_6932/3538882253.py:8: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`



## Summary table

In [31]:
cai = pd.DataFrame()
cai.index = chosen_tokens.index
cai['name'] = chosen_tokens['name']
cai['symbol'] = chosen_tokens['symbol']
cai['market_cap'] = chosen_tokens['market_cap']
cai['price'] = chosen_tokens['current_price']
cai['weight'] = adjusted_weights
cai['weight_converted'] = converted_weights
cai['address'] = chosen_tokens['addr']

cai = cai[cai['weight'] > 0]
cai = cai.sort_values("market_cap", ascending=False)
cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,3.450144e+09,9.730000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,8.938757e+07,0.261539,0.5,128,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,7.074822e+07,10.770000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


In [32]:
# Add assets and weights to database
db_assets = db_funcs.convert_to_sql_strings(list(cai.index))
db_weights = list(cai['weight'])
db_funcs.insert_values(db_key_date,db_assets,db_weights,db_benchmark_table)

In [33]:
#Querying current contract to ascertain which assets will be dropped

cai_token_address = '0x48f88A3fE843ccb0b5003e70B4192c1d7448bEf0'
ia_contract = web3_object.eth.contract(address=cai_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()


for address in addresses:
    address = address.lower()
    if address not in list(cai['address']):
        data = tokens_market_data[tokens_market_data['addr']== address]
        cai.loc[data.index[0]] = [data['name'][0],data['symbol'][0],data['market_cap'][0],data['current_price'][0],0,0,address]

cai

,name,symbol,market_cap,price,weight,weight_converted,address
id,,,,,,,
avalanche-2,Avalanche,AVAX,3.450144e+09,9.730000,0.2,51,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
joe,JOE,JOE,8.938757e+07,0.261539,0.5,128,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd
benqi-liquid-staked-avax,BENQI Liquid Staked AVAX,SAVAX,7.074822e+07,10.770000,0.3,76,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be


In [34]:
zipped_assets = list(zip(list(cai['address']),list(cai['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,0x6e84a6216ea6dacc71ee8e6b0a5b7322eebc0fdd,0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7
[76, 128, 51]
